In [12]:
import pandas as pd
import re
from datetime import datetime
import os

In [13]:
# Fungsi untuk membersihkan dataset dari kolom Unnamed
def bersihkan_dataset(df):
    # Identifikasi kolom Unnamed
    kolom_unnamed = [col for col in df.columns if 'Unnamed' in col]
    
    # Hapus kolom Unnamed
    if kolom_unnamed:
        print(f"Menghapus {len(kolom_unnamed)} kolom Unnamed")
        df = df.drop(columns=kolom_unnamed)
    
    return df

In [14]:
# Fungsi untuk memastikan format dataset seragam
def standarisasi_dataset(df, sumber):
    # Pastikan kolom penting ada
    kolom_penting = ['Title', 'Content']
    for kolom in kolom_penting:
        if kolom not in df.columns:
            df[kolom] = ''
    
    # Tambahkan kolom sumber berita
    df['Sumber'] = sumber
    
    # Pilih hanya kolom yang diperlukan
    kolom_yang_dipertahankan = kolom_penting + ['Sumber']
    if 'Category' in df.columns:
        kolom_yang_dipertahankan.append('Category')
    if 'Link' in df.columns:
        kolom_yang_dipertahankan.append('Link')
    if 'Created_at' in df.columns:
        kolom_yang_dipertahankan.append('Created_at')
    
    # Filter dataset agar hanya berisi kolom yang diperlukan
    df_standar = df[kolom_yang_dipertahankan].copy()
    
    return df_standar

In [15]:
# Fungsi untuk mengelompokkan berita berdasarkan faktor ekonomi
def kelompokkan_berita_faktor(df):
    # Definisikan kata kunci untuk setiap faktor
    faktor_keywords = {
        'suku_bunga': [
            'suku bunga', 'bunga', 'bi rate', 'bi 7-day repo rate', 'federal reserve', 'the fed', 
            'bank sentral', 'bank indonesia', 'bi', 'kenaikan suku bunga', 'penurunan suku bunga',
            'kebijakan moneter', 'moneter', 'bps', 'bunga acuan', 'bunga bank', 'fomc',
            'bank', 'pinjaman', 'kredit', 'deposito', 'tapering', 'likuiditas', 'quantitative easing',
            'fed rate', 'fed funds rate', 'rapat dewan gubernur', 'rdg', 'lps', 'dpk', 'yield', 'reverse repo'
        ],
        'impor': [
            'impor', 'barang impor', 'nilai impor', 'volume impor', 'komoditas impor', 
            'aktivitas impor', 'kebijakan impor', 'pajak impor', 'bea masuk', 'larangan impor',
            'pembatasan impor', 'neraca perdagangan', 'defisit perdagangan', 'bea cukai',
            'barang masuk', 'kebijakan impor', 'produk impor', 'kuota impor', 'tarif impor',
            'importir', 'barang modal', 'bahan baku impor', 'impor migas', 'impor non migas'
        ],
        'ekspor': [
            'ekspor', 'barang ekspor', 'nilai ekspor', 'volume ekspor', 'komoditas ekspor', 
            'perdagangan ekspor', 'kebijakan ekspor', 'pajak ekspor', 'insentif ekspor', 
            'dorongan ekspor', 'surplus perdagangan', 'ekspansi pasar', 'produk ekspor',
            'komoditas ekspor', 'pertumbuhan ekspor', 'pasar ekspor', 'eksportir',
            'kinerja ekspor', 'tujuan ekspor', 'ekspor migas', 'ekspor non migas', 'bea keluar'
        ]
    }
    
    # Fungsi untuk mendeteksi faktor dalam teks
    def deteksi_faktor(teks):
        if not isinstance(teks, str):
            return []
        
        teks_lower = teks.lower()
        faktor_terdeteksi = []
        
        for faktor, keywords in faktor_keywords.items():
            for keyword in keywords:
                if keyword in teks_lower:
                    faktor_terdeteksi.append(faktor)
                    break
        
        return faktor_terdeteksi
    
    # Menambahkan kolom gabungan untuk analisis
    df['teks_gabungan'] = df['Title'].fillna('') + ' ' + df['Content'].fillna('')
    
    # Mendeteksi faktor
    df['faktor_terdeteksi'] = df['teks_gabungan'].apply(deteksi_faktor)
    
    # Membuat kolom terpisah untuk setiap faktor (1 jika terdeteksi, 0 jika tidak)
    for faktor in faktor_keywords.keys():
        df[faktor] = df['faktor_terdeteksi'].apply(lambda x: 1 if faktor in x else 0)
    
    # Menambahkan kolom jumlah faktor
    df['jumlah_faktor'] = df[['suku_bunga', 'impor', 'ekspor']].sum(axis=1)
    
    return df

In [16]:
# Fungsi utama untuk memproses dataset dari kedua sumber
def proses_pengelompokkan_multi_sumber(file_path_detik, file_path_cnbc, hapus_tidak_relevan=True):
    try:
        # Proses dataset Detik
        print("Memproses dataset Detik...")
        df_detik = baca_dataset(file_path_detik)
        if df_detik is None:
            df_detik = buat_dataset_contoh('detik')
        
        df_detik = bersihkan_dataset(df_detik)
        df_detik = standarisasi_dataset(df_detik, 'Detik')
        
        # Proses dataset CNBC
        print("\nMemproses dataset CNBC...")
        df_cnbc = baca_dataset(file_path_cnbc)
        if df_cnbc is None:
            df_cnbc = buat_dataset_contoh('cnbc')
        
        df_cnbc = bersihkan_dataset(df_cnbc)
        df_cnbc = standarisasi_dataset(df_cnbc, 'CNBC')
        
        # Gabungkan kedua dataset
        print("\nMenggabungkan dataset...")
        df_gabungan = pd.concat([df_detik, df_cnbc], ignore_index=True)
        print(f"Total berita setelah penggabungan: {len(df_gabungan)}")
        
        # Simpan jumlah berita awal untuk statistik
        jumlah_berita_awal = len(df_gabungan)
        jumlah_detik = len(df_detik)
        jumlah_cnbc = len(df_cnbc)
        
        # Lakukan pengelompokkan faktor ekonomi
        print("\nMelakukan pengelompokkan berdasarkan faktor ekonomi...")
        df_hasil = kelompokkan_berita_faktor(df_gabungan)
        
        # Menampilkan ringkasan hasil sebelum penghapusan
        print("\nRingkasan Hasil Pengelompokkan (Sebelum Penghapusan):")
        print(f"Total berita: {len(df_hasil)}")
        print(f"  - Dari Detik: {jumlah_detik} ({jumlah_detik/len(df_hasil)*100:.1f}%)")
        print(f"  - Dari CNBC: {jumlah_cnbc} ({jumlah_cnbc/len(df_hasil)*100:.1f}%)")
        print(f"Berita terkait suku bunga: {df_hasil['suku_bunga'].sum()} ({df_hasil['suku_bunga'].sum()/len(df_hasil)*100:.1f}%)")
        print(f"Berita terkait impor: {df_hasil['impor'].sum()} ({df_hasil['impor'].sum()/len(df_hasil)*100:.1f}%)")
        print(f"Berita terkait ekspor: {df_hasil['ekspor'].sum()} ({df_hasil['ekspor'].sum()/len(df_hasil)*100:.1f}%)")
        
        # Distribusi berdasarkan sumber
        print("\nDistribusi faktor berdasarkan sumber berita:")
        for sumber in ['Detik', 'CNBC']:
            df_sumber = df_hasil[df_hasil['Sumber'] == sumber]
            if len(df_sumber) > 0:
                print(f"\n{sumber}:")
                print(f"  Total berita: {len(df_sumber)}")
                print(f"  Berita terkait suku bunga: {df_sumber['suku_bunga'].sum()} ({df_sumber['suku_bunga'].sum()/len(df_sumber)*100:.1f}%)")
                print(f"  Berita terkait impor: {df_sumber['impor'].sum()} ({df_sumber['impor'].sum()/len(df_sumber)*100:.1f}%)")
                print(f"  Berita terkait ekspor: {df_sumber['ekspor'].sum()} ({df_sumber['ekspor'].sum()/len(df_sumber)*100:.1f}%)")
        
        # Hitung berita yang terkait dengan beberapa faktor
        multi_count = (df_hasil['jumlah_faktor'] > 1).sum()
        print(f"\nBerita terkait dengan lebih dari satu faktor: {multi_count} ({multi_count/len(df_hasil)*100:.1f}%)")
        
        # Hitung berita yang tidak terkait dengan faktor apapun
        no_factor_count = (df_hasil['jumlah_faktor'] == 0).sum()
        print(f"Berita tidak terkait dengan faktor apapun: {no_factor_count} ({no_factor_count/len(df_hasil)*100:.1f}%)")
        
        # Menghapus berita yang tidak terkait dengan faktor apapun jika diminta
        if hapus_tidak_relevan:
            df_hasil_filtered = df_hasil[df_hasil['jumlah_faktor'] > 0].copy()
            print(f"\nMenghapus {no_factor_count} berita yang tidak terkait dengan faktor apapun...")
        else:
            df_hasil_filtered = df_hasil.copy()
            print("\nMempertahankan semua berita termasuk yang tidak terkait dengan faktor apapun...")
        
        # Menampilkan ringkasan hasil setelah penghapusan (jika ada)
        if hapus_tidak_relevan:
            print("\nRingkasan Hasil Pengelompokkan (Setelah Penghapusan):")
            print(f"Total berita: {len(df_hasil_filtered)} (dari {jumlah_berita_awal} berita awal)")
            
            if len(df_hasil_filtered) > 0:  # Untuk menghindari division by zero
                # Distribusi berdasarkan sumber setelah penghapusan
                detik_setelah = len(df_hasil_filtered[df_hasil_filtered['Sumber'] == 'Detik'])
                cnbc_setelah = len(df_hasil_filtered[df_hasil_filtered['Sumber'] == 'CNBC'])
                
                print(f"  - Dari Detik: {detik_setelah} ({detik_setelah/len(df_hasil_filtered)*100:.1f}%)")
                print(f"  - Dari CNBC: {cnbc_setelah} ({cnbc_setelah/len(df_hasil_filtered)*100:.1f}%)")
                print(f"Berita terkait suku bunga: {df_hasil_filtered['suku_bunga'].sum()} ({df_hasil_filtered['suku_bunga'].sum()/len(df_hasil_filtered)*100:.1f}%)")
                print(f"Berita terkait impor: {df_hasil_filtered['impor'].sum()} ({df_hasil_filtered['impor'].sum()/len(df_hasil_filtered)*100:.1f}%)")
                print(f"Berita terkait ekspor: {df_hasil_filtered['ekspor'].sum()} ({df_hasil_filtered['ekspor'].sum()/len(df_hasil_filtered)*100:.1f}%)")
        
        # Simpan hasil ke file CSV baru
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # Buat direktori hasil jika belum ada
        os.makedirs('hasil_pengelompokkan', exist_ok=True)
        
        # Simpan data lengkap (termasuk yang tidak relevan)
        output_file_full = f'hasil_pengelompokkan/hasil_gabungan_lengkap_{timestamp}.csv'
        df_hasil.to_csv(output_file_full, index=False)
        print(f"\nHasil pengelompokkan lengkap telah disimpan ke {output_file_full}")
        
        # Simpan data yang sudah difilter (jika pilihan hapus_tidak_relevan aktif)
        if hapus_tidak_relevan:
            output_file_filtered = f'hasil_pengelompokkan/hasil_gabungan_filtered_{timestamp}.csv'
            df_hasil_filtered.to_csv(output_file_filtered, index=False)
            print(f"Hasil pengelompokkan tanpa berita tidak relevan telah disimpan ke {output_file_filtered}")
            
            # Gunakan dataset yang sudah difilter untuk langkah selanjutnya
            df_aktif = df_hasil_filtered
        else:
            # Gunakan dataset lengkap
            df_aktif = df_hasil
        
        # Membuat file terpisah untuk setiap faktor dari dataset aktif
        for faktor in ['suku_bunga', 'impor', 'ekspor']:
            df_faktor = df_aktif[df_aktif[faktor] == 1]
            if len(df_faktor) > 0:
                faktor_file = f'hasil_pengelompokkan/berita_{faktor}_{timestamp}.csv'
                df_faktor.to_csv(faktor_file, index=False)
                print(f"Berita terkait {faktor} telah disimpan ke {faktor_file}")
                
                # Simpan juga berdasarkan sumber
                for sumber in ['Detik', 'CNBC']:
                    df_faktor_sumber = df_faktor[df_faktor['Sumber'] == sumber]
                    if len(df_faktor_sumber) > 0:
                        faktor_sumber_file = f'hasil_pengelompokkan/berita_{faktor}_{sumber.lower()}_{timestamp}.csv'
                        df_faktor_sumber.to_csv(faktor_sumber_file, index=False)
                        print(f"Berita {sumber} terkait {faktor} telah disimpan ke {faktor_sumber_file}")
        
        return df_aktif
    
    except Exception as e:
        print(f"Terjadi kesalahan: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

In [17]:
# Fungsi utama untuk memproses dataset dari kedua sumber
def proses_pengelompokkan_multi_sumber(file_path_detik, file_path_cnbc, hapus_tidak_relevan=True):
    try:
        # Proses dataset Detik
        print("Memproses dataset Detik...")
        df_detik = baca_dataset(file_path_detik)
        if df_detik is None:
            df_detik = buat_dataset_contoh('detik')
        
        df_detik = bersihkan_dataset(df_detik)
        df_detik = standarisasi_dataset(df_detik, 'Detik')
        
        # Proses dataset CNBC
        print("\nMemproses dataset CNBC...")
        df_cnbc = baca_dataset(file_path_cnbc)
        if df_cnbc is None:
            df_cnbc = buat_dataset_contoh('cnbc')
        
        df_cnbc = bersihkan_dataset(df_cnbc)
        df_cnbc = standarisasi_dataset(df_cnbc, 'CNBC')
        
        # Gabungkan kedua dataset
        print("\nMenggabungkan dataset...")
        df_gabungan = pd.concat([df_detik, df_cnbc], ignore_index=True)
        print(f"Total berita setelah penggabungan: {len(df_gabungan)}")
        
        # Simpan jumlah berita awal untuk statistik
        jumlah_berita_awal = len(df_gabungan)
        jumlah_detik = len(df_detik)
        jumlah_cnbc = len(df_cnbc)
        
        # Lakukan pengelompokkan faktor ekonomi
        print("\nMelakukan pengelompokkan berdasarkan faktor ekonomi...")
        df_hasil = kelompokkan_berita_faktor(df_gabungan)
        
        # Menampilkan ringkasan hasil sebelum penghapusan
        print("\nRingkasan Hasil Pengelompokkan (Sebelum Penghapusan):")
        print(f"Total berita: {len(df_hasil)}")
        print(f"  - Dari Detik: {jumlah_detik} ({jumlah_detik/len(df_hasil)*100:.1f}%)")
        print(f"  - Dari CNBC: {jumlah_cnbc} ({jumlah_cnbc/len(df_hasil)*100:.1f}%)")
        print(f"Berita terkait suku bunga: {df_hasil['suku_bunga'].sum()} ({df_hasil['suku_bunga'].sum()/len(df_hasil)*100:.1f}%)")
        print(f"Berita terkait impor: {df_hasil['impor'].sum()} ({df_hasil['impor'].sum()/len(df_hasil)*100:.1f}%)")
        print(f"Berita terkait ekspor: {df_hasil['ekspor'].sum()} ({df_hasil['ekspor'].sum()/len(df_hasil)*100:.1f}%)")
        
        # Distribusi berdasarkan sumber
        print("\nDistribusi faktor berdasarkan sumber berita:")
        for sumber in ['Detik', 'CNBC']:
            df_sumber = df_hasil[df_hasil['Sumber'] == sumber]
            if len(df_sumber) > 0:
                print(f"\n{sumber}:")
                print(f"  Total berita: {len(df_sumber)}")
                print(f"  Berita terkait suku bunga: {df_sumber['suku_bunga'].sum()} ({df_sumber['suku_bunga'].sum()/len(df_sumber)*100:.1f}%)")
                print(f"  Berita terkait impor: {df_sumber['impor'].sum()} ({df_sumber['impor'].sum()/len(df_sumber)*100:.1f}%)")
                print(f"  Berita terkait ekspor: {df_sumber['ekspor'].sum()} ({df_sumber['ekspor'].sum()/len(df_sumber)*100:.1f}%)")
        
        # Hitung berita yang terkait dengan beberapa faktor
        multi_count = (df_hasil['jumlah_faktor'] > 1).sum()
        print(f"\nBerita terkait dengan lebih dari satu faktor: {multi_count} ({multi_count/len(df_hasil)*100:.1f}%)")
        
        # Hitung berita yang tidak terkait dengan faktor apapun
        no_factor_count = (df_hasil['jumlah_faktor'] == 0).sum()
        print(f"Berita tidak terkait dengan faktor apapun: {no_factor_count} ({no_factor_count/len(df_hasil)*100:.1f}%)")
        
        # Menghapus berita yang tidak terkait dengan faktor apapun jika diminta
        if hapus_tidak_relevan:
            df_hasil_filtered = df_hasil[df_hasil['jumlah_faktor'] > 0].copy()
            print(f"\nMenghapus {no_factor_count} berita yang tidak terkait dengan faktor apapun...")
        else:
            df_hasil_filtered = df_hasil.copy()
            print("\nMempertahankan semua berita termasuk yang tidak terkait dengan faktor apapun...")
        
        # Menampilkan ringkasan hasil setelah penghapusan (jika ada)
        if hapus_tidak_relevan:
            print("\nRingkasan Hasil Pengelompokkan (Setelah Penghapusan):")
            print(f"Total berita: {len(df_hasil_filtered)} (dari {jumlah_berita_awal} berita awal)")
            
            if len(df_hasil_filtered) > 0:  # Untuk menghindari division by zero
                # Distribusi berdasarkan sumber setelah penghapusan
                detik_setelah = len(df_hasil_filtered[df_hasil_filtered['Sumber'] == 'Detik'])
                cnbc_setelah = len(df_hasil_filtered[df_hasil_filtered['Sumber'] == 'CNBC'])
                
                print(f"  - Dari Detik: {detik_setelah} ({detik_setelah/len(df_hasil_filtered)*100:.1f}%)")
                print(f"  - Dari CNBC: {cnbc_setelah} ({cnbc_setelah/len(df_hasil_filtered)*100:.1f}%)")
                print(f"Berita terkait suku bunga: {df_hasil_filtered['suku_bunga'].sum()} ({df_hasil_filtered['suku_bunga'].sum()/len(df_hasil_filtered)*100:.1f}%)")
                print(f"Berita terkait impor: {df_hasil_filtered['impor'].sum()} ({df_hasil_filtered['impor'].sum()/len(df_hasil_filtered)*100:.1f}%)")
                print(f"Berita terkait ekspor: {df_hasil_filtered['ekspor'].sum()} ({df_hasil_filtered['ekspor'].sum()/len(df_hasil_filtered)*100:.1f}%)")
        
        # Simpan hasil ke file CSV baru
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # Buat direktori hasil jika belum ada
        os.makedirs('hasil_pengelompokkan', exist_ok=True)
        
        # Simpan data lengkap (termasuk yang tidak relevan)
        output_file_full = f'hasil_pengelompokkan/hasil_gabungan_lengkap_{timestamp}.csv'
        df_hasil.to_csv(output_file_full, index=False)
        print(f"\nHasil pengelompokkan lengkap telah disimpan ke {output_file_full}")
        
        # Simpan data yang sudah difilter (jika pilihan hapus_tidak_relevan aktif)
        if hapus_tidak_relevan:
            output_file_filtered = f'hasil_pengelompokkan/hasil_gabungan_filtered_{timestamp}.csv'
            df_hasil_filtered.to_csv(output_file_filtered, index=False)
            print(f"Hasil pengelompokkan tanpa berita tidak relevan telah disimpan ke {output_file_filtered}")
            
            # Gunakan dataset yang sudah difilter untuk langkah selanjutnya
            df_aktif = df_hasil_filtered
        else:
            # Gunakan dataset lengkap
            df_aktif = df_hasil
        
        # Membuat file terpisah untuk setiap faktor dari dataset aktif
        for faktor in ['suku_bunga', 'impor', 'ekspor']:
            df_faktor = df_aktif[df_aktif[faktor] == 1]
            if len(df_faktor) > 0:
                faktor_file = f'hasil_pengelompokkan/berita_{faktor}_{timestamp}.csv'
                df_faktor.to_csv(faktor_file, index=False)
                print(f"Berita terkait {faktor} telah disimpan ke {faktor_file}")
                
                # Simpan juga berdasarkan sumber
                for sumber in ['Detik', 'CNBC']:
                    df_faktor_sumber = df_faktor[df_faktor['Sumber'] == sumber]
                    if len(df_faktor_sumber) > 0:
                        faktor_sumber_file = f'hasil_pengelompokkan/berita_{faktor}_{sumber.lower()}_{timestamp}.csv'
                        df_faktor_sumber.to_csv(faktor_sumber_file, index=False)
                        print(f"Berita {sumber} terkait {faktor} telah disimpan ke {faktor_sumber_file}")
        
        return df_aktif
    
    except Exception as e:
        print(f"Terjadi kesalahan: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

In [18]:
# Fungsi untuk membaca dataset dengan berbagai encoding
def baca_dataset(file_path):
    try:
        # Coba berbagai metode pembacaan file
        df = None
        encodings = ['utf-8', 'latin1', 'cp1252']
        
        for encoding in encodings:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                print(f"Berhasil membaca file dengan encoding {encoding}")
                print(f"Jumlah berita: {len(df)}")
                print(f"Kolom-kolom: {', '.join(df.columns[:5])}...")
                return df
            except UnicodeDecodeError:
                continue
            except Exception as e:
                print(f"Error: {str(e)}")
                continue
        
        print(f"Tidak dapat membaca file: {file_path}")
        return None
        
    except Exception as e:
        print(f"Error saat membaca file {file_path}: {str(e)}")
        return None

In [19]:
# Fungsi untuk membuat dataset contoh jika file tidak dapat dibaca
def buat_dataset_contoh(sumber):
    if sumber.lower() == 'detik':
        contoh_data = {
            'Category': ['finance', 'economy', 'business'],
            'Title': [
                'BI Naikkan Suku Bunga 25 Basis Poin',
                'Nilai Impor Indonesia Turun 5% di Kuartal I',
                'Ekspor Komoditas Pertanian Meningkat Tajam'
            ],
            'Content': [
                'Bank Indonesia memutuskan untuk menaikkan suku bunga acuan sebesar 25 basis poin menjadi 5,75% dalam Rapat Dewan Gubernur.',
                'Nilai impor Indonesia mengalami penurunan sebesar 5% pada kuartal I 2023 dibandingkan periode yang sama tahun lalu.',
                'Ekspor komoditas pertanian Indonesia meningkat tajam sebesar 15% pada semester I 2023.'
            ]
        }
    else:  # cnbc
        contoh_data = {
            'Category': ['finance', 'economy', 'business'],
            'Title': [
                'The Fed Pertahankan Suku Bunga, Rupiah Menguat',
                'Neraca Perdagangan Surplus Berkat Lonjakan Ekspor',
                'Presiden Resmikan Gedung Baru di Jakarta'
            ],
            'Content': [
                'Bank Sentral Amerika Serikat (The Fed) memutuskan untuk mempertahankan suku bunga. Rupiah menguat terhadap dolar AS.',
                'Indonesia mencatat surplus neraca perdagangan berkat lonjakan nilai ekspor yang melebihi nilai impor.',
                'Presiden meresmikan gedung baru di Jakarta. Acara dihadiri oleh sejumlah menteri kabinet.'
            ]
        }
    
    print(f"Membuat dataset contoh untuk {sumber}")
    return pd.DataFrame(contoh_data)

In [20]:
# Jalankan fungsi utama
if __name__ == "__main__":
    # Ganti dengan path file CSV yang sebenarnya
    file_path_detik = 'D:\SKRIPSI\skripsi_2025\dataset berita terdahulu\detik_news_scraped (2) - detik_news_scraped (2).csv'
    file_path_cnbc = 'D:\SKRIPSI\skripsi_2025\dataset berita terdahulu\cnbc_news_scraped (3) - cnbc_news_scraped (3).csv'
    
    # Set hapus_tidak_relevan=True untuk menghapus berita yang tidak terkait faktor ekonomi
    hasil = proses_pengelompokkan_multi_sumber(file_path_detik, file_path_cnbc, hapus_tidak_relevan=True)

Memproses dataset Detik...
Berhasil membaca file dengan encoding utf-8
Jumlah berita: 3863
Kolom-kolom: Category, Title, Description, Content, Link...
Menghapus 192 kolom Unnamed

Memproses dataset CNBC...
Berhasil membaca file dengan encoding utf-8
Jumlah berita: 2868
Kolom-kolom: Category, Title, Content, Link, Created_at...
Menghapus 5 kolom Unnamed

Menggabungkan dataset...
Total berita setelah penggabungan: 6731

Melakukan pengelompokkan berdasarkan faktor ekonomi...

Ringkasan Hasil Pengelompokkan (Sebelum Penghapusan):
Total berita: 6731
  - Dari Detik: 3863 (57.4%)
  - Dari CNBC: 2868 (42.6%)
Berita terkait suku bunga: 6239 (92.7%)
Berita terkait impor: 1001 (14.9%)
Berita terkait ekspor: 859 (12.8%)

Distribusi faktor berdasarkan sumber berita:

Detik:
  Total berita: 3863
  Berita terkait suku bunga: 3511 (90.9%)
  Berita terkait impor: 267 (6.9%)
  Berita terkait ekspor: 225 (5.8%)

CNBC:
  Total berita: 2868
  Berita terkait suku bunga: 2728 (95.1%)
  Berita terkait impor: 